In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import word2vec

# スコア評価用ライブラリ
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support

# エラー表示の抑制
import warnings

In [2]:
# 表示関連
# DataFrameの列数設定
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_rows", 500)
warnings.filterwarnings('ignore')

In [45]:
# テキストデータの読み込み
wikiData = pd.read_csv("csv/train.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [46]:
# 数字の除外
wikiData['text'] = wikiData['text'].str.replace('[0-9]+','', regex=True)

# 不要な記号の除外
path = "data/Removal Symbols.txt"
removal_symbols = []
with open(path, encoding="utf-8") as f:
    removal_symbols = [s.strip() for s in f.readlines()]
print(removal_symbols)

for symbol in removal_symbols:
    wikiData['text'] = wikiData['text'].str.replace(symbol,'')

['(', ')', '（', '）', '"', '""', '「', '」', '[', ']', '', '', '']


In [47]:
# Numpy配列へ変換
X = wikiData.text.values
y = wikiData.target.values

In [48]:
# トレーニングデータ、テストデータの分割
X_train, X_test, y_train, y_test = train_test_split(X , y, test_size=0.2, random_state=42, stratify=y)

In [49]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] in ['名詞'] :
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [50]:
# 全テキストの結合
allText = ''
for text in wikiData['text'].values:
    allText += ''.join(text)

In [51]:
# 形態素解析
words = tokenize(allText)

In [56]:
# Word2Vecモデルの学習
model = word2vec.Word2Vec(words, size=100, min_count=5, window=5, iter=100)
model.save("word2vec.gensim.model")

In [54]:
print(model.__dict__['wv']['株式会社'])

KeyError: "word '株式会社' not in vocabulary"

In [ ]:
# Light GBM
import lightgbm as lgb

clf = lgb.LGBMClassifier(objective='multiclass', num_class=33)
clf.fit(train_matrix, y_train)
cv_scores = cross_val_score(clf, train_matrix, y_train, cv=5)

print("Training score：" + str(clf.score(train_matrix, y_train)))
print("Cross-Validation score：" + str(np.mean(cv_scores)))
print("Test score：" + str(clf.score(test_matrix, y_test)))